## Import thư viện

In [51]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from transformers import AutoTokenizer
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import classification_report

## Load dataset

In [52]:
dataset = load_dataset("conll2003")
dataset['train'][0]

c:\Users\solit\anaconda3\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [53]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [54]:
label_list = dataset["train"].features["ner_tags"].feature.names
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for label, idx in label_to_id.items()}
label_to_id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [55]:
class NERDataset(Dataset):
    def __init__(self, data, tokenizer, label_to_id, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.label_to_id = label_to_id
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data[idx]["tokens"]
        labels = self.data[idx]["ner_tags"]

        encoding = self.tokenizer(tokens, truncation=True, is_split_into_words=True, 
                                  padding="max_length", max_length=self.max_len, return_tensors="pt")
        word_ids = encoding.word_ids()

        aligned_labels = [-100 if word_id is None else labels[word_id] for word_id in word_ids]

        return encoding["input_ids"].squeeze(0), encoding["attention_mask"].squeeze(0), torch.tensor(aligned_labels)

In [56]:
train_dataset = NERDataset(dataset["train"], tokenizer, label_to_id)
val_dataset = NERDataset(dataset["validation"], tokenizer, label_to_id)
test_dataset = NERDataset(dataset["test"], tokenizer, label_to_id)

## Model

In [57]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        embeddings = self.embedding(input_ids)
        packed_output, _ = self.lstm(embeddings)
        logits = self.fc(packed_output)
        return logits

## Train, test

In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [59]:
embed_dim = 128
hidden_dim = 256
num_labels = len(label_to_id)
vocab_size = len(tokenizer)
batch_size = 64
num_epochs = 50

In [60]:
model = LSTMModel(vocab_size, embed_dim, hidden_dim, num_labels)
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [61]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def train(model, train_loader, criterion, optimizer, num_epochs=100):
    model.train()
    for epoch in tqdm(range(num_epochs)):
        running_loss = 0
        for input_ids, attention_mask, labels in train_loader:
            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = criterion(logits.view(-1, num_labels), labels.view(-1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/100:.4f}')

def evaluate(model, dataloader, label_to_id):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for input_ids, attention_mask, labels in dataloader:
            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=-1)
            all_preds.extend(predictions.view(-1).tolist())
            all_labels.extend(labels.view(-1).tolist())

    all_preds = [p for p, l in zip(all_preds, all_labels) if l != -100]
    all_labels = [l for l in all_labels if l != -100]
    print(classification_report(all_labels, all_preds, target_names=list(label_to_id.keys())))

In [63]:
train(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

  2%|▏         | 1/50 [00:57<46:40, 57.15s/it]

Epoch [1/50], Loss: 1.5411


  4%|▍         | 2/50 [01:52<45:03, 56.33s/it]

Epoch [2/50], Loss: 0.7716


  6%|▌         | 3/50 [02:48<43:58, 56.14s/it]

...

Epoch [48/50], Loss: 0.0007


 98%|█████████▊| 49/50 [45:41<00:55, 55.56s/it]

Epoch [49/50], Loss: 0.0005


100%|██████████| 50/50 [46:37<00:00, 55.95s/it]

Epoch [50/50], Loss: 0.0004


In [64]:
evaluate(model, test_loader, label_to_id)

              precision    recall  f1-score   support

           O       0.96      0.98      0.97     47914
       B-PER       0.73      0.74      0.74      2986
       I-PER       0.86      0.81      0.84      2704
       B-ORG       0.77      0.71      0.74      3524
       I-ORG       0.73      0.68      0.71      1309
       B-LOC       0.85      0.82      0.83      2998
       I-LOC       0.77      0.64      0.70       415
      B-MISC       0.63      0.60      0.62      1266
      I-MISC       0.46      0.54      0.50       322

    accuracy                           0.92     63438
   macro avg       0.75      0.72      0.74     63438
weighted avg       0.92      0.92      0.92     63438

